Exercise 7.1: The Forward-Backward algorithm

In [1]:
import numpy as np

# Define the HMM parameters
states = [1, 2, 3, 4, 5]
vocab = [
    "a",
    "the",
    "over",
    "beside",
    "near",
    "quick",
    "brown",
    "lazy",
    "jumps",
    "runs",
    "walks",
    "fox",
    "dog",
    "cat",
]
beta = [
    [0.6, 0.4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0.2, 0.4, 0.4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0.5, 0.3, 0.2, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0.1, 0.4, 0.5, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3, 0.4, 0.3],
]
theta = [
    [0, 0, 0.5, 0, 0.5],
    [1, 0, 0, 0, 0],
    [0, 0, 0.3, 0, 0.7],
    [0, 1, 0, 0, 0],
    [0, 0.5, 0, 0.5, 0],
]
initial_probs = np.array([0.2, 0.2, 0.2, 0.2, 0.2])  # Uniform distribution

In [2]:
# Define the observation sequence
obs_seq = ['the', 'quick', 'fox', 'jumps', 'over', 'a', 'dog']
obs_seq_indices = [vocab.index(obs) for obs in obs_seq]
print(obs_seq_indices)

[1, 5, 11, 8, 2, 0, 12]


In [3]:
# Initialize the forward and backward variables
N = len(states)
T = len(obs_seq)
print(N)
print(T)
# Initialize the chosen variables
t = 4

5
7


In [4]:
def forward_backward_algorithm(beta, theta, initial_probs, states, obs_seq_indices):
    # Initialize the forward and backward variables
    N = len(states)
    T = len(obs_seq_indices)
    forward = np.zeros((N, T))
    backward = np.zeros((N, T))

    # Forward pass
    for t in range(T):
        for i in range(N):
            if t == 0:
                forward[i, t] = initial_probs[i] * beta[i][obs_seq_indices[t]]
            else:
                forward[i, t] = beta[i][obs_seq_indices[t]] * np.sum(
                    [forward[j, t - 1] * theta[j][i] for j in range(N)]
                )

    # Backward pass
    for t in range(T - 1, -1, -1):
        for i in range(N):
            if t == T - 1:
                backward[i, t] = 1
            else:
                backward[i, t] = np.sum(
                    [theta[i][j] * beta[j][obs_seq_indices[t + 1]] * backward[j, t + 1] for j in range(N)]
                )

    # Calculate the probability
    probability = np.sum([forward[i, T - 1] * backward[i, T - 1] for i in range(N)])

    return probability


In [5]:
# Calculate the probability
probability = forward_backward_algorithm(beta, theta, initial_probs, states, obs_seq_indices)
print(probability)

5.040000000000002e-06


Exercise 7.2: The Viterbi algorithm

In [6]:
# Define the HMM parameters
states = [1, 2, 3, 4, 5]
vocab = [
    "a",
    "the",
    "I",
    "you",
    "can",
    "will",
    "call",
    "own",
    "take",
    "book",
    "round",
    "claim",
    "car",
    "hotel",
    "new",
    "great",
]
beta = [
    [0.4, 0.3, 0.2, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0.3, 0.4, 0, 0.3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0.1, 0.15, 0.15, 0.15, 0.15, 0.1, 0.1, 0.1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3, 0, 0, 0, 0.4, 0.3],
    [0, 0, 0, 0.05, 0.05, 0.05, 0.1, 0, 0, 0.15, 0.1, 0.15, 0.2, 0.15, 0, 0],
]
theta = [
    [0, 0, 0.3, 0.3, 0.4],
    [0, 0, 0, 0.4, 0.6],
    [0, 0.75, 0.25, 0, 0],
    [0, 0, 0, 0, 1],
    [0, 0, 1, 0, 0],
]
initial_probs = np.array([0.2, 0.2, 0.2, 0.2, 0.2])  # Uniform distribution

# Define the observation sequence
obs_seq = ["you", "claim", "you", "can", "book", "a", "round", "hotel"]
obs_seq_indices = [vocab.index(obs) for obs in obs_seq]
print(obs_seq_indices)

[3, 11, 3, 4, 9, 0, 10, 13]


In [7]:
# Initialize the Viterbi variables
N = len(states)
T = len(obs_seq)
viterbi = np.zeros((N, T))
backpointer = np.zeros((N, T), dtype=int)

# Initialization
for s in range(N):
    viterbi[s, 0] = initial_probs[s] * beta[s][obs_seq_indices[0]]
print(N)
print(T)

5
8


In [8]:
# Recursion
for t in range(1, T):
    for s in range(N):
        viterbi[s, t] = max(
            [
                viterbi[s_prime, t - 1]
                * theta[s_prime][s]
                * beta[s][obs_seq_indices[t]]
                for s_prime in range(N)
            ]
        )
        backpointer[s, t] = np.argmax(
            [viterbi[s_prime, t - 1] * theta[s_prime][s] for s_prime in range(N)]
        )

# Termination
best_path_prob = np.max(viterbi[:, T - 1])
best_path_pointer = np.argmax(viterbi[:, T - 1])

# Backtrack to get the best path
best_path = []
for t in range(T - 1, -1, -1):
    best_path.insert(0, best_path_pointer)
    best_path_pointer = backpointer[best_path_pointer, t]

print("Best path:", best_path)
print("Best path probability:", best_path_prob)

Best path: [4, 2, 1, 4, 2, 1, 3, 4]
Best path probability: 2.7337500000000004e-09


Exercise 7.3: The Baum-Welch algorithm

In [9]:
# Initialize the HMM parameters from the problem
initial_distribution = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
transition_matrix = np.array(
    [
        [0, 0, 0.5, 0, 0.5],
        [1, 0, 0, 0, 0],
        [0, 0, 0.3, 0, 0.7],
        [0, 1, 0, 0, 0],
        [0, 0.5, 0, 0.5, 0],
    ]
)
emission_matrix = np.array(
    [
        [0.6, 0.4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0.2, 0.4, 0.4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0.5, 0.3, 0.2, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0.1, 0.4, 0.5, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3, 0.4, 0.3],
    ]
)
word_to_index = {
    word: index
    for index, word in enumerate(
        [
            "a",
            "the",
            "over",
            "beside",
            "near",
            "quick",
            "brown",
            "lazy",
            "jumps",
            "runs",
            "walks",
            "fox",
            "dog",
            "cat",
        ]
    )
}
observed_sequence = [
    word_to_index[word] for word in "the quick fox jumps over a dog".split()
]

N = initial_distribution.shape[0]
T = len(observed_sequence)

Updated transition matrix: [[0.00e+00 0.00e+00 3.15e-05 0.00e+00 1.00e-01]
 [1.20e-01 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 2.52e-04]
 [0.00e+00 2.40e-02 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 1.20e-03 0.00e+00]]
Updated emission matrix: [[0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0. ]]


In [ ]:
# Initialize the forward probabilities matrix
forward_probabilities = np.zeros((N, T))

# Initialize the forward probabilities for the first observation
for i in range(N):
    forward_probabilities[i, 0] = (
        initial_distribution[i] * emission_matrix[i, observed_sequence[0]]
    )

# Compute the forward probabilities for the rest of the observations
for t in range(1, T):
    for j in range(N):
        forward_probabilities[j, t] = (
            np.sum(forward_probabilities[:, t - 1] * transition_matrix[:, j])
            * emission_matrix[j, observed_sequence[t]]
        )
# Initialize the backward probabilities matrix
backward_probabilities = np.zeros((N, T))

# Initialize the backward probabilities for the last observation
backward_probabilities[:, -1] = 1

# Compute the backward probabilities for the rest of the observations
for t in range(T - 2, -1, -1):
    for i in range(N):
        backward_probabilities[i, t] = np.sum(
            backward_probabilities[:, t + 1]
            * transition_matrix[i, :]
            * emission_matrix[:, observed_sequence[t + 1]]
        )
# Compute the posterior probabilities
posterior_probabilities = (
    forward_probabilities
    * backward_probabilities
    / np.sum(forward_probabilities[:, -1])
)

# Initialize the new transition matrix and emission matrix
new_transition_matrix = np.zeros((N, N))
new_emission_matrix = np.zeros((N, len(word_to_index)))

# Update the transition matrix and emission matrix
for i in range(N):
    for j in range(N):
        new_transition_matrix[i, j] = np.sum(
            posterior_probabilities[i, :-1]
            * transition_matrix[i, j]
            * emission_matrix[j, observed_sequence[1:]]
            * backward_probabilities[j, 1:]
        ) / np.sum(posterior_probabilities[i, :-1])
    for w in range(len(word_to_index)):
        new_emission_matrix[i, w] = np.sum(
            posterior_probabilities[i, np.array(observed_sequence) == w]
        ) / np.sum(posterior_probabilities[i, :])

# The new transition matrix and emission matrix are the updated parameters
print("Updated transition matrix:", new_transition_matrix)
print("Updated emission matrix:", new_emission_matrix)

Exercise 7.4: Argue with an AI chatbot about hidden Markov models

Phrase: hidden Markov models and the Viterbi and Baum-Welch algorithms are still important and
needed in the age of neural large language models
 try to argue for the opposite position

CHAT GPT RESPONSE
While hidden Markov models (HMMs) and the associated algorithms like Viterbi and Baum-Welch have been fundamental in various applications, arguing for their diminished relevance in the age of neural large language models can be done by highlighting the strengths and advancements of neural models. Here are some points to consider:

End-to-End Learning:

Neural language models, especially deep learning models like transformers, enable end-to-end learning. They can directly map inputs to outputs without relying on intermediate representations like states in HMMs. This simplifies the modeling process and allows for more direct and seamless learning.
Representation Learning:

Neural language models excel at learning complex hierarchical representations of data. They automatically learn meaningful features and representations from the input data, eliminating the need for manual feature engineering. In contrast, HMMs may require careful design of features and assumptions about the underlying structure of the data.
Scalability:

Neural language models can handle large datasets and scale effectively with the increasing availability of computational resources. As data sizes grow, the scalability of neural models becomes a significant advantage compared to HMMs, which may face challenges in handling large datasets efficiently.
Task-Specific Architectures:

Neural language models can be designed and fine-tuned for specific tasks without the need for a generic model like an HMM. Task-specific architectures can outperform generic models in terms of accuracy and efficiency, providing a more tailored solution for different applications.
Complexity of Language:

Natural language is highly complex, and neural models have shown an ability to capture intricate patterns and semantics in data. HMMs may struggle to capture the diversity and nuances present in language compared to the expressive power of neural architectures.
Training Efficiency:

Training neural models often involves end-to-end backpropagation, which can be more efficient than the iterative algorithms like Baum-Welch used in training HMMs. The parallelization capabilities of neural network training contribute to faster convergence on large datasets.
Transfer Learning:

Pre-trained neural language models support transfer learning, where models trained on a large corpus can be fine-tuned for specific tasks with limited data. This can be advantageous in scenarios where labeled data is scarce, allowing for more effective adaptation to new domains.
While HMMs and their associated algorithms have been essential in certain contexts, the capabilities of neural large language models often overshadow their traditional counterparts in terms of flexibility, scalability, and end-to-end learning. It's essential to consider the specific requirements of a task when choosing between these approaches.


Phrase: hidden Markov models and the Viterbi and Baum-Welch algorithms are still important and
needed in the age of neural large language models
argue for a particular opinion (important/not
important) and argue yourself for the opposite position

ChatGPT RESPONSE

"
Arguing for the Importance of Hidden Markov Models (HMMs), Viterbi, and Baum-Welch Algorithms in the Age of Neural Large Language Models:

Robustness and Interpretability:

HMMs offer a structured probabilistic framework that can provide a level of interpretability lacking in black-box neural models. This is crucial in applications where understanding the underlying process is essential.
Limited Data Scenarios:

In situations with limited data, which can occur in niche domains or when working with specific languages or dialects, HMMs can be more effective due to their ability to make reasonable assumptions about the underlying structure of the data.
Sequential Data Modeling:

HMMs are designed to model sequential data, making them well-suited for tasks such as speech recognition, gesture recognition, and natural language processing where the order of observations is crucial. Neural models might struggle with capturing long-range dependencies in sequential data.
Training Efficiency:

The training process for HMMs, especially with algorithms like Baum-Welch, can be more computationally efficient than training large neural models, especially when dealing with smaller datasets. This efficiency can be crucial in real-time applications.
Transfer Learning and Hybrid Models:

Combining HMMs with neural models can offer a hybrid approach that benefits from the strengths of both paradigms. For instance, pre-training an HMM on a related task and fine-tuning with a neural model can be a powerful strategy in scenarios where large labeled datasets are not readily available.
Arguing for the Diminished Importance of HMMs, Viterbi, and Baum-Welch Algorithms in the Age of Neural Large Language Models:

End-to-End Learning:

Neural models, especially large language models, are designed for end-to-end learning, eliminating the need for manually engineering features or specifying explicit state transitions as in HMMs. This makes them more adaptable to diverse tasks.
Representation Learning:

Neural models excel at learning hierarchical representations of data, automatically capturing intricate patterns and dependencies. HMMs, on the other hand, might struggle to capture complex relationships present in large and diverse datasets.
Data Abundance:

In the age of big data, where massive labeled datasets are available, neural models can leverage this abundance to learn intricate patterns and nuances, potentially outperforming HMMs that might require more manual intervention and domain expertise.
Scalability:

Neural models can scale to handle vast amounts of data and parameters, making them more suitable for contemporary tasks that involve massive datasets, such as machine translation, sentiment analysis, and question answering.
Flexibility:

Neural models are inherently more flexible and adaptable to various domains and tasks. Their architectures can be easily modified or fine-tuned for specific applications, providing a level of versatility that might be challenging for HMMs to match.
In conclusion, while HMMs and their associated algorithms still hold importance in certain contexts, the rise of neural large language models has reshaped the landscape, emphasizing the need for adaptability, scalability, and end-to-end learning capabilities. The choice between these approaches ultimately depends on the specific requirements and constraints of the given task or application
"

Arguments for Suitability of Markov Model Assumptions:

Simplicity: Assumes simplicity in state transitions, making it easy to model and interpret.

Memoryless: Appropriate for certain systems where future states depend only on the current state, simplifying modeling.



Arguments Against Suitability of Markov Model Assumptions:

Fixed Order: Assumes fixed order of dependencies, limiting its ability to capture complex, varying relationships.

Fixed State: Might struggle in situations where dependencies are influenced by more than just the current state.

Limited Context: Fails to consider broader context which can be crucial in certain applications.